## 🔢 Description Crawler

In [1]:
# Imports
import numpy    as np
import pandas   as pd
import google_play_scraper 
import langdetect
import datetime
import requests
import time
import json
import os

#### Initialization

In [2]:
print("⚡ Start - {} ⚡\n".format(datetime.datetime.now()))
startTime = datetime.datetime.now()

⚡ Start - 2024-07-22 17:18:05.759761 ⚡



#### 📥 1) Load Data 

In [3]:
DATA_PATH = "../../0_Data/4_MudFlow.csv"

# Read the data
appsDF = pd.read_csv(DATA_PATH)
appsDF = appsDF.drop(columns=['classID'])

# TEST
appsDF = appsDF.head(3)

print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

--- #️⃣ Apps: 3 


In [4]:
appsDF.head(5)

,sha256,pkgName
0,093EBD1938A3FD17C91FBB34AED59E53F927EDBCE36865...,com.zoner.android.security
1,B0842156E992486357E98E029BA9126D9C6312C658DBC7...,com.grgbanking.card
2,CFC732676EA8FCC9C3CBF8AEA806BEFB4E6F00D8ECE10C...,com.markeu.msscan


GP

In [5]:
# def getGoogleEnglishDescription(pkgName):
# 	try:
# 		# Use googlePlayScraper Library
# 		result = google_play_scraper.app(pkgName, lang='en', country='us')

# 		if result is not None and result['description'] is not None:
# 			try:
# 				# Use langdetect
# 				lang = langdetect.detect(result['description'])
# 				# If the description is in English, return it
# 				if lang == "en":
# 					description = result['description'].replace('\n', ' ').replace('\r', '')
# 					return pd.Series([description], index=['description'])
# 				else:
# 					return pd.Series([np.nan], index=['description'])
# 			except langdetect.LangDetectException:
# 				return pd.Series([np.nan], index=['description'])
# 		else:
# 			return pd.Series([np.nan], index=['description'])
# 	except Exception:
# 		return pd.Series([np.nan], index=['description'])

In [6]:
# appsDF['gpDescription'] = appsDF['pkgName'].apply(getGoogleEnglishDescription)
# appsDF.head(3)

AZ

In [9]:
def getDescriptionFromMetaDataAndroZoo(pkgName, retries=10, delay=20):
	url = 'https://androzoo.uni.lu/api/get_gp_metadata/{}'.format(pkgName)
	params = {'apikey': os.getenv('ANDROZOO_API_KEY')}
	
	print("--- 📦 PKG NAME: {}".format(pkgName))
	
	attempt = 0
	while attempt < retries:
		print(f"--- ▶️ Attempt #{attempt}")
		response = requests.get(url, params=params)
		
		# Return Description
		if response.status_code == 200:
			print(response.json)
			return response.json()[0]['descriptionHtml']
		# Retry 
		elif response.status_code in [502, 503, 400]:
			attempt += 1
			print(f"--- ❌ Attempt {attempt} failed with status code {response.status_code}. Retrying in {delay} seconds...")
			time.sleep(delay)
		else:
			print(f"--- ❌ Request failed with status code {response.status_code}. No retries left.")
			return None
	
	print("--- ☹️ Max retries exceeded. Request failed.")
	return None

In [10]:
appsDF['azDescription'] = appsDF['pkgName'].apply(getDescriptionFromMetaDataAndroZoo)
appsDF.head(3)

--- 📦 PKG NAME: com.zoner.android.security
--- ▶️ Attempt #0
--- ❌ Attempt 1 failed with status code 502. Retrying in 20 seconds...
--- ▶️ Attempt #1
--- ❌ Attempt 2 failed with status code 502. Retrying in 20 seconds...
--- ▶️ Attempt #2
<bound method Response.json of <Response [200]>>


IndexError: list index out of range

### Filter the data.

In [ ]:
# # Create the 'description' column
# appsDF['googlePlayDescription'] = appsDF['gpDescription'].fillna(appsDF['azDescription'])
# print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

# # Drop rows where both 'gpDescription' and 'azDescription' are NaN
# appsDF = appsDF.dropna(subset=['description'])
# print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

# appsDF = appsDF.drop(columns=['gpDescription', 'azDescription'])
# appsDF.head(3)

In [ ]:
appsDF['googlePlayDescription'] = appsDF['gpDescription']
print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

appsDF = appsDF.dropna(subset=['googlePlayDescription'])
print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

appsDF = appsDF.drop(columns=['gpDescription'])
appsDF.head(3)

In [ ]:
appsDF.to_csv("../TmpData/4_MudFlow_Description.csv", index=False)

##### 🔚 End

In [ ]:
endTime = datetime.datetime.now()
print("\n🔚 --- End - {} --- 🔚".format(endTime))

# Assuming endTime and startTime are in seconds
totalTime = endTime - startTime
minutes = totalTime.total_seconds() // 60
seconds = totalTime.total_seconds() % 60
print("⏱️ --- Time: {:02d} minutes and {:02d} seconds --- ⏱️".format(int(minutes), int(seconds)))